## Setting ()

In [5]:
# ! pip install nibabel
# ! pip install SimpleITK

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Looking in indexes: http://ftp.daumkakao.com/pypi/simple
     |████████████████████████████████| 48.4 MB 8.9 MB/s eta 0:00:01    |██                              | 3.0 MB 8.9 MB/s eta 0:00:06     |███████████████████████████▎    | 41.2 MB 8.9 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import nibabel as nib
from typing import List
from collections import OrderedDict
import SimpleITK as sitk
import random
import pandas as pd
from tqdm import notebook

In [2]:
# Utils ..
def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)
    
def subdirs(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isdir(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

def subfiles(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isfile(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

def img_size(file_path: str = '/mnt/dataset/test_set'):


    file_dir = file_path

    data_imgs = subfiles(file_dir, suffix='png')
    shape_list = []

    
    for i in data_imgs:
        pngTemp = plt.imread(i)

        shap = pngTemp.shape

        shape_list.append(shap)


    shape_list.sort()
    shape_list_unq = np.unique(shape_list, axis=0)
    print(file_path, ', Unique Size : ')
    print(shape_list_unq)

    return shape_list_unq

In [3]:
maic_dir = '/tf/backup/'
base_dir = os.path.join(maic_dir, 'working')
input_dir = '/mnt/dataset/test_set/'
temp_dir = '/tf/temp/'

maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

# ! git clone https://github.com/keemsir/nnUNet.git

respository_dir = os.path.join(maic_dir, 'nnUNet')
os.chdir(respository_dir)

! pip install -e .

os.chdir(base_dir)

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
Obtaining file:///tf/backup/nnUNet
  Attempting uninstall: nnunet
    Found existing installation: nnunet 1.7.0
    Can't uninstall 'nnunet'. No files were found to uninstall.
  Running setup.py develop for nnunet
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
task_name = 'Task99_Inference'
'''
Task90_GRSR
Task91_NDDR
Task92_BFCR
Task93_MSCR
Task94_SIRL
Task95_GRSL
Task96_NDHL
Task97_CLALM3MT
Task98_CLALUCUK

Task99_Inference
'''
main_dir = os.path.join(base_dir, 'nnUNet/nnunet')
mainT_dir = os.path.join(temp_dir, 'nnUNet/nnunet')

rawbase_dir = os.path.join(mainT_dir, 'nnUNet_raw_data_base/')

pp_dir = os.path.join(mainT_dir, 'preprocessed')
tasks_dir = os.path.join(mainT_dir, 'Tasks')
task_dir = os.path.join(tasks_dir, task_name)

model_dir = os.path.join(main_dir, 'nnUNet_trained_models')

# 1. Data preprocessing
maybe_mkdir_p(tasks_dir)
maybe_mkdir_p(temp_dir)

# 2. Directory
maybe_mkdir_p(main_dir)
maybe_mkdir_p(model_dir)
maybe_mkdir_p(pp_dir)

# 3. Directory
# maybe_mkdir_p(result_dir)
# maybe_mkdir_p(staple_dir)


#Environment Setting
os.environ['nnUNet_raw_data_base'] = rawbase_dir #os.path.join(mainT_dir, 'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = pp_dir #os.path.join(mainT_dir, 'preprocessed')
os.environ['RESULTS_FOLDER'] = model_dir #os.path.join(main_dir, 'nnUNet_trained_models')


print('Setting Completed!')

Setting Completed!


## Testdataset Preprocessing

In [9]:
name_shape_DIC_ = {}
for i in subfiles('/mnt/dataset/test_set/', join=False, suffix='.png'):
    i_path = os.path.join('/mnt/dataset/test_set/', i)
    shape_ = plt.imread(i_path).shape
    i_, _ = os.path.splitext(i)
    name_shape_DIC_.update({i_:shape_})

In [5]:
def png2nifti_test(test_folder: str = '/mnt/dataset/test_set/', 
                   save_folder: str = '/tf/temp/nnUNet/nnunet/Tasks/Task99_Inference'):

    # surgery_NUM : surgery_DIC Num (0~8)
    # training_folder : training file path '/mnt/dataset/train-valid_set'
    # test_folder : test file path '/mnt/dataset/test_set'
    # save_folder : '/tf/temp/nnUNet/nnunet/Tasks/' [imagesTr, imagesTs] Save Folder path

    test_filelist = subfiles(test_folder, join=False, suffix='.png')

    maybe_mkdir_p(os.path.join(save_folder, 'imagesTr'))
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTs'))
    maybe_mkdir_p(os.path.join(save_folder, 'labelsTr'))
    
    print('Creating "{}" Images for Inference ..'.format(test_folder))
    
    
    list_test_1 = test_filelist[0:100]
    list_test_2 = test_filelist[100:200]
    list_test_3 = test_filelist[200:300]
    list_test_4 = test_filelist[300:400]
    list_test_5 = test_filelist[400:500]
    list_test_6 = test_filelist[500:600]
    list_test_7 = test_filelist[600:700]
    list_test_8 = test_filelist[700:800]
    list_test_9 = test_filelist[800:900]
    list_test_10 = test_filelist[900:1000]
    list_test_11 = test_filelist[1000:1100]
    list_test_12 = test_filelist[1100:1200]
    list_test_13 = test_filelist[1200:1300]
    list_test_14 = test_filelist[1300:1400]
    
    list_list = [list_test_1, list_test_2, list_test_3, list_test_4, list_test_5, list_test_6, list_test_7,
                 list_test_8, list_test_9, list_test_10, list_test_11, list_test_12, list_test_13, list_test_14]

    
    name_shape_DIC = {}
    
    for list_ in notebook.tqdm(list_list):
        
        list_NAME = str(list_)
        images = np.zeros([512, 512, 0], dtype=np.single)
    
        for tf in notebook.tqdm(list_):

            # Initial Setting

            test_path = os.path.join(test_folder, tf)


            # print(test_path)

            png_data = plt.imread(test_path)
            png_data_shape = png_data.shape
            tf_, _ = os.path.splitext(tf)
            name_shape_DIC.update({tf_:png_data_shape})


            png_data = np.mean(png_data, axis=2)
            png_data = cv2.resize(png_data, dsize=(512, 512), interpolation=cv2.INTER_CUBIC) # size reduce

            image_png = np.expand_dims(png_data, axis=2)

            images = np.append(images, image_png, axis=2)
        
        tf_name, _ = os.path.splitext(tf)
        
        niim = nib.Nifti1Image(images, affine=np.eye(4))
        nib.save(niim, os.path.join(save_folder, 'imagesTs/{}.nii.gz'.format(tf_name)))
        print(tf_name, images.shape)
        del images
    
    print('"{}" Images for Inference Completed !!'.format(save_folder))
    
    return name_shape_DIC

In [7]:
name_shape = png2nifti_test()

Creating "/mnt/dataset/test_set/" Images for Inference ..


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

img_00100 (512, 512, 100)


  0%|          | 0/100 [00:00<?, ?it/s]

...

img_01200 (512, 512, 100)


  0%|          | 0/100 [00:00<?, ?it/s]

img_01300 (512, 512, 100)


  0%|          | 0/80 [00:00<?, ?it/s]

img_01380 (512, 512, 80)
"/tf/temp/nnUNet/nnunet/Tasks/Task99_Inference" Images for Inference Completed !!


In [12]:
def json_mk(save_dir: str):
    # Path
    imagesTr = os.path.join(save_dir, 'imagesTr')
    imagesTs = os.path.join(save_dir, 'imagesTs')
    maybe_mkdir_p(imagesTr)
    maybe_mkdir_p(imagesTs)
    
    overwrite_json_file = True
    json_file_exist = False
    
    if os.path.exists(os.path.join(save_dir, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True
        
    if json_file_exist == False or overwrite_json_file:
        
        json_dict = OrderedDict()
        json_dict['name'] = "SUR"
        json_dict['description'] = "Robotic Surgery AI Challenge 2021"
        json_dict['tensorImageSize'] = "3D"
        json_dict['reference'] = "https://maic.or.kr/"
        json_dict['licence'] = "SNUH"
        json_dict['release'] = "24/11/2021"
        
        json_dict['modality'] = {
            "0": "png"
        }
        json_dict['labels'] = {
            "0": "background",
            "1": "surgery"
        }

        train_ids = subfiles(imagesTr, join=False, suffix='.nii.gz')
        test_ids = subfiles(imagesTs, join=False, suffix='.nii.gz')
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

        json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids] # (i[:i.find("_0000")])

        with open(os.path.join(save_dir, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=False)

        if os.path.exists(os.path.join(save_dir, 'dataset.json')):
            if json_file_exist == False:
                print('{}/dataset.json <-- created!'.format(save_dir))
            else:
                print('{}/dataset.json <-- overwritten!'.format(save_dir))

In [13]:
json_mk('/tf/temp/nnUNet/nnunet/Tasks/Task99_Inference')

/tf/temp/nnUNet/nnunet/Tasks/Task99_Inference/dataset.json <-- created!


In [9]:
! nnUNet_convert_decathlon_task -i /tf/temp/nnUNet/nnunet/Tasks/Task99_Inference/ -output_task_id 799 # -i : task_dir



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



## File path

In [37]:
print('Model list :', os.listdir('/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/'))
print('\nraw_data list', os.listdir('/tf/temp/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/'))
print('\nPredicted list', os.listdir('/tf/backup/predict/'))

Model list : ['Task484_SIRL', 'Task480_GRSR', 'Task487_CLALM3MT', 'Task486_NDHL', 'Task488_CLALUCUK', '.ipynb_checkpoints', 'Task483_MSCR', 'Task482_BFCR', 'Task485_GRSL', 'Task481_NDDR']

raw_data list ['Task482_BFCR', 'Task799_Inference', 'Task487_CLALM3MT', 'Task488_CLALUCUK', 'Task483_MSCR']

Predicted list ['Task484_SIRL', 'Task480_GRSR', 'Task486_NDHL', '.ipynb_checkpoints', 'Task482_BFCR', 'Task485_GRSL', 'Task481_NDDR']


## Prediction

In [45]:
!nnUNet_predict -i /tf/temp/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task799_Inference/imagesTs -o /tf/backup/predict/Task488_CLALUCUK -t 488 -tr nnUNetTrainerV2 -m 2d



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task488_CLALUCUK/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 1 input modalities for each image
Found 14 unique case ids, here are some examples: ['img_00800' 'img_00700' 'img_00800' 'img_00700' 'img_00900' 'img_00300'
 'img_00600' 'img_01200' 'img_01100' 'img_01200']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 14
number of cases that still need to be predicted: 14
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look

In [128]:
name_shape
for key in name_shape:
    print(key)

img_00001
img_00002
img_00003
img_00004
img_00005
img_00006
img_00007
img_00008
img_00009
img_00010
img_00011
img_00012
img_00013
img_00014
img_00015
img_00016
img_00017
img_00018
img_00019
img_00020
img_00021
img_00022
img_00023
img_00024
img_00025
img_00026
img_00027
img_00028
img_00029
img_00030
img_00031
img_00032
img_00033
img_00034
img_00035
img_00036
img_00037
img_00038
img_00039
img_00040
img_00041
img_00042
img_00043
img_00044
img_00045
img_00046
img_00047
img_00048
img_00049
img_00050
img_00051
img_00052
img_00053
img_00054
img_00055
img_00056
img_00057
img_00058
img_00059
img_00060
img_00061
img_00062
img_00063
img_00064
img_00065
img_00066
img_00067
img_00068
img_00069
img_00070
img_00071
img_00072
img_00073
img_00074
img_00075
img_00076
img_00077
img_00078
img_00079
img_00080
img_00081
img_00082
img_00083
img_00084
img_00085
img_00086
img_00087
img_00088
img_00089
img_00090
img_00091
img_00092
img_00093
img_00094
img_00095
img_00096
img_00097
img_00098
img_00099
img_00100


In [ ]:
print(subfiles('/tf/temp/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task790_GRSR/imagesTs'))
print(subfiles('/tf/backup/predict/Task480_GRSR'))

def predict_view(NUM:int, test_arr1:str, test_arr2:str):

    plt.figure(figsize=(15,30))
    
    image_arr1 = np.array(nib.load((test_arr1)).dataobj)
    image_arr2 = np.array(nib.load(test_arr2).dataobj)

    plt.subplot(1,2,1)
    plt.imshow(image_arr1[:, :,NUM])
    plt.axis('off')
    
    plt.subplot(1,2,2)    
    plt.imshow(image_arr2[:, :, NUM])
    plt.axis('off')


    plt.show()

## Submission

In [52]:
ima__ = '/tf/backup/predict/Task480_GRSR/img_01100.nii.gz'

ima_arr__ = np.array(nib.load(ima__).dataobj)
ima_arr__.shape

(512, 512, 100)

In [90]:
data = {'file' : [img_name], 'class' : [class_NUM], 'x1' : [x_1], 'y1' : [y_1],
        'x2' : [x_2], 'y2' : [y_2], 'x3' : [x_3], 'y3' : [y_3],
        'x4' : [x_4], 'y4' : [y_4], 'x5' : [x_5], 'y5' : [y_5],
        'x6' : [x_6], 'y6' : [y_6], 'x7' : [x_7], 'y7' : [y_7],
        'x8' : [x_8], 'y8' : [y_8], 'x9' : [x_9], 'y9' : [y_9], 'x10' : [x_10], 'y10' : [y_10]}
df = pd.DataFrame(data)

NameError: name 'img_name' is not defined

In [16]:
vv__, ll__ = findMaxVolumeAndPoints(ima_arr__[:,:,1])
ll__

NameError: name 'ima_arr__' is not defined

In [11]:
from skimage.measure import label, regionprops
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors
import networkx as nx

def findMaxVolumeAndPoints(mask_):
    # The largest area
    props = regionprops(label(np.float64(np.asanyarray(mask_))))
    areaList = []
    for prop in props:
        areaList.append(prop.area)
#         print(prop.area)
    

    maxIndex = np.argmax(areaList)
    targetProp = props[maxIndex]
#     # debug
#     plt.figure()
#     plt.imshow(targetProp.image)
    
    # New image generation
    refinedImg = np.zeros(np.shape(mask_))
    refinedImg[targetProp.bbox[0]:targetProp.bbox[2], targetProp.bbox[1]:targetProp.bbox[3]] = targetProp.convex_image
    refinedImg = np.uint8(refinedImg)
    # edge image and their coordinates
    edges = cv2.Canny(refinedImg, 0, 1)
#     plt.figure()
#     plt.imshow(edges)
    indices = np.where(edges != [0])
    coordinates = np.column_stack((indices[0], indices[1]))

    # point order optm.
    x = indices[0]
    y = indices[1]   
    points = np.c_[x, y]

    #clf = NearestNeighbors(2).fit(points)
    neigh = NearestNeighbors(n_neighbors=3, radius=0.5)
    clf = neigh.fit(points)
    G = clf.kneighbors_graph()
    T = nx.from_scipy_sparse_matrix(G)

    paths = [list(nx.dfs_preorder_nodes(T, i)) for i in range(len(points))]
    mindist = np.inf
    minidx = 0

    for i in range(len(points)):
        p = paths[i]  # order of nodes
        ordered = points[p]  # ordered nodes
        # find cost of that order by the sum of euclidean distances between points (i) and (i+1)
        cost = (((ordered[:-1] - ordered[1:]) ** 2).sum(1)).sum()
        if cost < mindist:
            mindist = cost
            minidx = i

    opt_order = paths[minidx]
    xx = x[opt_order]
    yy = y[opt_order]

    numPoints = 10
    stepSizeTemp = np.uint8(np.round(len(xx)/numPoints))

    xxExtracted = xx[::stepSizeTemp]
    yyExtracted = yy[::stepSizeTemp]
    
    if len(xxExtracted)==11:
        xxExtracted = list(xxExtracted)
        yyExtracted = list(yyExtracted)
        xxExtracted.pop()
        yyExtracted.pop()
        xxExtracted = tuple(xxExtracted)
        yyExtracted = tuple(yyExtracted)
        
        
        
    coordinatesF = np.column_stack((xxExtracted, yyExtracted))
    volumeF = targetProp.area

#     plt.figure()
#     plt.plot(coordinatesF[:, 0], coordinatesF[:, 1])
#     plt.plot(xx, yy)
#     plt.show()

    return volumeF, coordinatesF

In [14]:
name_shape_list = list(name_shape.keys())
name_shape_list[0]

'img_00001'

In [16]:
def submission_task(Task_input: str):
    
    task_NameDIC = {'Task480_GRSR': 0, 
                    'Task481_NDDR': 1, 
                    'Task482_BFCR': 2, 
                    'Task483_MSCR': 3, 
                    'Task484_SIRL': 4, 
                    'Task485_GRSL': 5, 
                    'Task486_NDHL': 6, 
                    'Task487_CLALM3MT': 7, 
                    'Task488_CLALUCUK': 8}
    
    Task_Threshold = {'Task480_GRSR': 0.011768341064453125*0.9,
                 'Task481_NDDR': 0.007961273193359375*0.9,
                 'Task482_BFCR': 0.010486602783203125*0.9,
                 'Task483_MSCR': 0.00832366943359375*0.9,
                 'Task484_SIRL': 0.004627227783203125*0.9,
                 'Task485_GRSL': 0.00714874267578125*0.9,
                 'Task486_NDHL': 0.009521484375*0.9,
                 'Task487_CLALM3MT': 0.0224609375*0.9,
                 'Task488_CLALUCUK': 0.023807525634765625*0.9}
    
    Pred_dir = '/tf/backup/predict/'
    Task_path = os.path.join(Pred_dir, Task_input)
    Task_list = subfiles(Task_path, suffix='.nii.gz', join=False)
    COUNT_iter = 0
    name_shape_list = list(name_shape.keys())
    
    for Tl_ in notebook.tqdm(Task_list):
        
        Tl_path = os.path.join(Task_path, Tl_)
        
        Tl_arr = np.array(nib.load(Tl_path).dataobj) # Tl_arr : (x,y,z) z : slice
        
        # cv2.resize(Tl_arr, dsize=(1980, 1980), interpolation=cv2.INTER_CUBIC)
        
        _, _, Tl_z = Tl_arr.shape
        
        
        for z_ in notebook.tqdm(range(Tl_z)):
            
            img_name = name_shape_list[COUNT_iter]
            
            dsize_x, dsize_y, _ = name_shape[img_name]
            
            Tl_arr_resize = cv2.resize(Tl_arr[:,:,z_], dsize=(dsize_x, dsize_y), interpolation=cv2.INTER_CUBIC)
            
            try:
            # if True:

                volume_f, Tl_arr_xy = findMaxVolumeAndPoints(Tl_arr_resize) # If add

                # print(volume_f/(dsize_x*dsize_y) , Task_Threshold[Task_input])
                if volume_f/(dsize_x*dsize_y) > Task_Threshold[Task_input]:

                    IMG_NAME = img_name

                    class_NUM = task_NameDIC[Task_input]

                    x_1, y_1 = Tl_arr_xy[0]
                    x_2, y_2 = Tl_arr_xy[1]
                    x_3, y_3 = Tl_arr_xy[2]
                    x_4, y_4 = Tl_arr_xy[3]
                    x_5, y_5 = Tl_arr_xy[4]
                    x_6, y_6 = Tl_arr_xy[5]
                    x_7, y_7 = Tl_arr_xy[6]
                    x_8, y_8 = Tl_arr_xy[7]
                    x_9, y_9 = Tl_arr_xy[8]
                    x_10, y_10 = Tl_arr_xy[9]


                    submission_Row = {'file' : [IMG_NAME], 'class' : [class_NUM], 'x1' : [x_1], 'y1' : [y_1],
                    'x2' : [x_2], 'y2' : [y_2], 'x3' : [x_3], 'y3' : [y_3],
                    'x4' : [x_4], 'y4' : [y_4], 'x5' : [x_5], 'y5' : [y_5],
                    'x6' : [x_6], 'y6' : [y_6], 'x7' : [x_7], 'y7' : [y_7],
                    'x8' : [x_8], 'y8' : [y_8], 'x9' : [x_9], 'y9' : [y_9], 'x10' : [x_10], 'y10' : [y_10]}

                    df = pd.DataFrame(submission_Row)

                    # .to_csv
                    if not os.path.exists('submission_{}.csv'.format(Task_input)):
                        df.to_csv('submission_{}.csv'.format(Task_input), index=False, mode='w')
                    else:
                        df.to_csv('submission_{}.csv'.format(Task_input), index=False, mode='a', header=False)
                
                # del IMG_NAME, class_NUM, x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4, x_5, y_5, x_6, y_6
                # del x_7, y_7, x_8, y_8, x_9, y_9, x_10, y_10

            except:
                print('Exception occurred. -> {}'.format(img_name))
                
                
            
            COUNT_iter += 1
            

        
        
    return None

In [17]:
submission_task('Task480_GRSR')

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Exception occurred. -> img_00183


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [18]:
submission_task('Task481_NDDR')

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Exception occurred. -> img_00251


...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
submission_task('Task482_BFCR')

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
submission_task('Task483_MSCR')

In [ ]:
submission_task('Task484_SIRL')

In [22]:
submission_task('Task485_GRSL')

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Exception occurred. -> img_00029
Exception occurred. -> img_00032
Exception occurred. -> img_00041
Exception occurred. -> img_00046
Exception occurred. -> img_00052
Exception occurred. -> img_00068
Exception occurred. -> img_00070
Exception occurred. -> img_00075
Exception occurred. -> img_00084


  0%|          | 0/100 [00:00<?, ?it/s]

Exception occurred. -> img_00105
Exception occurred. -> img_00115
Exception occurred. -> img_00127
Exception occurred. -> img_00132
Exception occurred. -> img_00147
Exception occurred. -> img_00154
Exception occurred. -> img_00155
Exception occurred. -> img_00161
Exception occurred. -> img_00175
Exception occurred. -> img_00180
Exception occurred. -> img_00188
Exception occurred. -> img_00190
Exception occurred. -> img_00192
Exception occurred. -> img_00193
Exception occurred. -> img_00194
Exception occurred. -> img_00197


...

Exception occurred. -> img_01114
Exception occurred. -> img_01116
Exception occurred. -> img_01124
Exception occurred. -> img_01136
Exception occurred. -> img_01142
Exception occurred. -> img_01143
Exception occurred. -> img_01144
Exception occurred. -> img_01160
Exception occurred. -> img_01173
Exception occurred. -> img_01175
Exception occurred. -> img_01185
Exception occurred. -> img_01186
Exception occurred. -> img_01194
Exception occurred. -> img_01195


  0%|          | 0/100 [00:00<?, ?it/s]

Exception occurred. -> img_01205
Exception occurred. -> img_01240
Exception occurred. -> img_01255
Exception occurred. -> img_01258
Exception occurred. -> img_01278
Exception occurred. -> img_01281
Exception occurred. -> img_01282
Exception occurred. -> img_01291


  0%|          | 0/80 [00:00<?, ?it/s]

Exception occurred. -> img_01311
Exception occurred. -> img_01314
Exception occurred. -> img_01324
Exception occurred. -> img_01328
Exception occurred. -> img_01340
Exception occurred. -> img_01358
Exception occurred. -> img_01365
Exception occurred. -> img_01369
Exception occurred. -> img_01375
Exception occurred. -> img_01376


In [23]:
submission_task('Task486_NDHL')

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
submission_task('Task487_CLALM3MT')

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Exception occurred. -> img_00063


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Exception occurred. -> img_00360


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
submission_task('Task488_CLALUCUK')

In [64]:
# Task_input : 'Task480_GRSR', 'Task481_NDDR', 'Task482_BFCR', 'Task483_MSCR', 'Task484_SIRL', 
#              'Task485_GRSL', 'Task486_NDHL', 'Task487_CLALM3MT', 'Task488_CLALUCUK'

# PN_list = os.listdir('./models/result')
# PN_list.sort()

img_list = subfiles('/mnt/dataset/test_set', join=False, suffix='.png')

for img_l in img_list:
    img_name, _ = os.path.splitext(img_l)



#         ctarr = np.array(nib.load('./models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0000.nii.gz'.format(pn)).dataobj)
#         pred_arr = np.array(nib.load('./models/result/{}.nii.gz'.format(pn)).dataobj)

    #Calculate Mean SUV and Max SUV
    
#     class_NUM = random.randrange(9)
    class_NUM = 'None'
    
    x_1 = 'None'
    x_2 = 'None'
    x_3 = 'None'
    x_4 = 'None'
    x_5 = 'None'
    x_6 = 'None'
    x_7 = 'None'
    x_8 = 'None'
    x_9 = 'None'
    x_10 = 'None'

    y_1 = 'None'
    y_2 = 'None'
    y_3 = 'None'
    y_4 = 'None'
    y_5 = 'None'
    y_6 = 'None'
    y_7 = 'None'
    y_8 = 'None'
    y_9 = 'None'
    y_10 = 'None'

#     suvmax, suvmean = get_suv_params(ptarr, pred_arr)


#         data = {'case' : [PATIENT_NUM], 'PD_Aorta_volume' : [aorvol], 'PD_SUVmean' : [suvmean]}
    data = {'file' : [img_name], 'class' : [class_NUM], 'x1' : [x_1], 'y1' : [y_1],
            'x2' : [x_2], 'y2' : [y_2], 'x3' : [x_3], 'y3' : [y_3],
            'x4' : [x_4], 'y4' : [y_4], 'x5' : [x_5], 'y5' : [y_5],
            'x6' : [x_6], 'y6' : [y_6], 'x7' : [x_7], 'y7' : [y_7],
            'x8' : [x_8], 'y8' : [y_8], 'x9' : [x_9], 'y9' : [y_9], 'x10' : [x_10], 'y10' : [y_10]}
    df = pd.DataFrame(data)

    # .to_csv
    if not os.path.exists('submission.csv'):
        df.to_csv('submission.csv', index=False, mode='w')
    else:
        df.to_csv('submission.csv', index=False, mode='a', header=False)


In [ ]:
os.listdir('/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/')

In [44]:
import shutil

for i in range(5):
    shutil.copytree('/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task488_CLALUCUK/nnUNetTrainerV2__nnUNetPlansv2.1/all', '/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task488_CLALUCUK/nnUNetTrainerV2__nnUNetPlansv2.1/fold_{}'.format(i))
